# Pandas to Dask to Coiled

We scale a simple ETL workflow from a single thread to the cloud

1.  Pandas on a sample of data
2.  Pandas on a complete file
3.  Dask + Pandas on a set of files in parallel
4.  Coiled + Dask + Pandas to run on the cloud

## Pandas: Convert CSV to Parquet

### Download the data from Amazon

### Investigate data locally with Pandas


In [1]:
import pandas as pd
df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-10.csv", 
    nrows=10000,
)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-10-01 00:19:55,2019-10-01 00:23:57,1,0.40,1,N,48,163,2,4.5,3.0,0.5,0.00,0.00,0.3,8.30,2.5
1,1,2019-10-01 00:40:19,2019-10-01 00:55:17,2,4.30,1,N,144,141,1,14.5,3.0,0.5,2.00,0.00,0.3,20.30,2.5
2,1,2019-10-01 00:06:52,2019-10-01 00:21:23,1,5.00,1,N,137,80,1,17.0,3.0,0.5,5.20,0.00,0.3,26.00,2.5
3,2,2019-10-01 00:36:08,2019-10-01 00:36:15,1,0.00,1,N,25,25,4,-2.5,-0.5,-0.5,0.00,0.00,-0.3,-3.80,0.0
4,2,2019-10-01 00:36:08,2019-10-01 00:36:15,1,0.00,1,N,25,25,2,2.5,0.5,0.5,0.00,0.00,0.3,3.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,2019-10-01 06:52:13,2019-10-01 07:03:36,2,3.21,1,N,161,231,2,11.0,0.0,0.5,0.00,0.00,0.3,14.30,2.5
9996,1,2019-10-01 06:51:25,2019-10-01 07:15:12,1,0.00,1,N,42,146,1,24.2,0.0,0.5,0.00,6.12,0.3,31.12,0.0
9997,2,2019-10-01 06:23:53,2019-10-01 06:26:30,1,0.52,1,N,263,263,2,4.0,0.0,0.5,0.00,0.00,0.3,7.30,2.5
9998,2,2019-10-01 06:41:37,2019-10-01 06:52:00,1,2.20,1,N,236,163,1,9.5,0.0,0.5,1.54,0.00,0.3,14.34,2.5


In [2]:
!head data_taxi/yellow_tripdata_2019-01.csv

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,.00,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,
2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,
2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,
2,2018-11-28 16:25:49,2018-11-28 16:28:26,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,5.76,0.3,13.31,
2,2018-11-28 16:29:37,2018-11-28 16:33:43,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,
1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.30,1,N,163,229,1,6.5,0.5,0.5,1.25,0,0.3,9.05,
1,2019-01-01 00:32:01,2019-01-01 0

### Massage data types 

Before we convert to parquet format, let's clean up our types a little

In [3]:
%%time

import pandas as pd

df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-01.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
)
df

CPU times: user 17.3 s, sys: 3.85 s, total: 21.1 s
Wall time: 21.4 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667787,2,2019-01-31 23:57:36,2019-02-01 00:18:39,1,4.79,1,N,263,4,1,18.0,0.5,0.5,3.86,0.0,0.3,23.16,0.0
7667788,2,2019-01-31 23:32:03,2019-01-31 23:33:11,1,0.00,1,N,193,193,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667789,2,2019-01-31 23:36:36,2019-01-31 23:36:40,1,0.00,1,N,264,264,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667790,2,2019-01-31 23:14:53,2019-01-31 23:15:20,1,0.00,1,N,264,7,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [4]:
%%time
df = df.astype({
    "VendorID": "uint8",
    "passenger_count": "uint8",
    "RatecodeID": "uint8",
    "store_and_fwd_flag": "category",
    "PULocationID": "uint16",
    "DOLocationID": "uint16",    
})

df["tip_ratio"] = df.tip_amount / df.total_amount

CPU times: user 1.28 s, sys: 1.68 s, total: 2.95 s
Wall time: 2.73 s


## Convert to Parquet

In [5]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019-01.parq")

CPU times: user 5.84 s, sys: 1.12 s, total: 6.96 s
Wall time: 5.67 s


In [6]:
%%time
df = pd.read_parquet("data_taxi/yellow_tripdata_2019-01.parq", columns=["passenger_count"])
df

CPU times: user 83.1 ms, sys: 94.4 ms, total: 177 ms
Wall time: 185 ms


,passenger_count
0,1
1,1
2,3
3,5
4,5
...,...
7667787,1
7667788,1
7667789,1
7667790,1


## Operate on many files in a for loop?

We could do this, but it's unpleasant

```python
for filename in glob("~/data/nyctaxi/yellow_tripdata_2019-*.parq"):
    df = pd.read_csv(filename)
    ...
    df.to_parquet(...)
```

## Use Dask locally to process the full dataset

In [7]:
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

/Users/hugobowne-anderson/opt/anaconda3/envs/data-science-at-scale/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49500 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:49501 Dashboard: http://127.0.0.1:49500/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [9]:
%%time

import dask.dataframe as dd

df = dd.read_csv(
    "~/Downloads/data-science-at-scale-master/data_taxi/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'}

)
df

CPU times: user 32.4 ms, sys: 7.47 ms, total: 39.9 ms
Wall time: 37 ms


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=117,,,,,,,,,,,,,,,,,,
,float64,datetime64[ns],datetime64[ns],float64,float64,float64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

df["tip_ratio"] = df.tip_amount / df.total_amount

In [11]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019.parq")

CPU times: user 1min, sys: 8.59 s, total: 1min 8s
Wall time: 4min 54s


## Optional: Work directly from the cloud with Coiled 

It will take about a minute to provision our resources on the cloud

In [12]:
import coiled

In [ ]:
coiled.create_software_environment(
    name="my-software-env",
    conda="binder/environment.yml",
)

In [30]:
# Control the resources of your cluster by creating a new cluster configuration
coiled.create_cluster_configuration(
    name="my-cluster-config",
    worker_memory="16 GiB",
    worker_cpu=4,
    scheduler_memory="8 GiB",
    scheduler_cpu=2,
    software="my-software-env",
)


In [31]:
import coiled
cluster = coiled.Cluster(n_workers=10, configuration="my-cluster-config")

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


In [32]:
client = Client(cluster)
client

distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://172.20.10.11:50090 remote=tls://ec2-3-12-85-20.us-east-2.compute.amazonaws.com:8786>


Client Scheduler: tls://ec2-3-12-85-20.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-3-12-85-20.us-east-2.compute.amazonaws.com:8787/status,Cluster Workers: 10 Cores: 40 Memory: 171.80 GB


In [33]:
import dask.dataframe as dd
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'
    },
    storage_options={"anon":True}
)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=127,,,,,,,,,,,,,,,,,,
,float64,datetime64[ns],datetime64[ns],float64,float64,float64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [34]:
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

df["tip_ratio"] = df.tip_amount / df.total_amount

In [35]:
%%time
df.to_parquet("yellow_tripdata_2019-coiled.parq")

KilledWorker: ("('read-csv-rename-dbdc4c74c6971cc3b4065c512dd1a615', 54)", <Worker 'tls://10.1.12.247:46755', name: hugobowne-678-worker-8-b0a898, memory: 0, processing: 32>)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [29]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
